## RNN介绍

RNN的目的使用来处理序列数据。在传统的神经网络模型中，是从输入层到隐含层再到输出层，层与层之间是全连接的，每层之间的节点是无连接的。但是这种普通的神经网络对于很多问题却无能无力。例如，你要预测句子的下一个单词是什么，一般需要用到前面的单词，因为一个句子中前后单词并不是独立的。RNNs之所以称为循环神经网路，即一个序列当前的输出与前面的输出也有关。具体的表现形式为网络会对前面的信息进行记忆并应用于当前输出的计算中，即隐藏层之间的节点不再无连接而是有连接的，并且隐藏层的输入不仅包括输入层的输出还包括上一时刻隐藏层的输出

In [1]:
import tensorflow as tf
import numpy as np

## MNIST数据集

###  加载MNIST数据集

In [3]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('E:\\src\\pycharm\\tensorflow-learn\\data\\mnist', one_hot=True)

Extracting E:\src\pycharm\tensorflow-learn\data\mnist\train-images-idx3-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\train-labels-idx1-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\t10k-images-idx3-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\t10k-labels-idx1-ubyte.gz


### 定义网络的参数
一张图片是 28*28，全连接神经网络是一次性把数据输入到网络，RNN是把它分为块

In [5]:
chunk_size = 28
chunk_n = 28

rnn_size = 256

n_output_layer = 10  # 输出层

### 获得数据集

In [13]:
X = tf.placeholder(tf.float32 , [None , chunk_n , chunk_size])  # 传入的是 chunk_n 为第二维度大小，chunk_size为第三维度大小，然后第一维度不定的三维tensor

Y = tf.placeholder(tf.float32)

## 定义待训练的神经网络

In [18]:
def recurrent_neural_network(data):
    layer = {'w_':tf.Variable(tf.random_normal([rnn_size, n_output_layer])),
             'b_':tf.Variable(tf.random_normal([n_output_layer]))}
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    data = tf.transpose(data, [1, 0, 2])
    data = tf.reshape(data, [-1, chunk_size])
    data = tf.split(data, chunk_n, 0)
    outputs, status = tf.contrib.rnn.static_rnn(lstm_cell, data, dtype=tf.float32)
    
    output = tf.add(tf.matmul(outputs[-1], layer['w_']), layer['b_'])
    
    return output

In [19]:
# 每次使用100条数据进行训练
batch_size = 100

## 使用数据集训练神经网络

In [22]:
def train_neural_network(X, Y):
    predict = recurrent_neural_network(X)
    cost_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))
    optimizer = tf.train.AdamOptimizer().minimize(cost_func)
    
    epochs = 13
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        epoch_loss = 0
        for epoch in range(epochs):
            for i in range(int(mnist.train.num_examples/batch_size)):
                x, y = mnist.train.next_batch(batch_size)
                x = x.reshape([batch_size, chunk_n, chunk_size])
                _, c = session.run([optimizer, cost_func], feed_dict={X:x,Y:y})
                epoch_loss += c
            print(epoch, ' : ', epoch_loss)
        
        correct = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('准确率：', accuracy.eval({X:mnist.test.images.reshape(-1, chunk_n, chunk_size), Y:mnist.test.labels}))

In [23]:
train_neural_network(X , Y)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0  :  201.80437238886952
1  :  264.90096345264465
2  :  306.96007743338123
3  :  340.337119553471
4  :  367.3596371212043
5  :  389.54029929009266
6  :  407.70144107274245
7  :  424.9165580555564
8  :  439.8533933736908
9  :  453.35339649439265
10  :  465.5640587847738
11  :  475.03265490471676
12  :  485.70960237595136
准确率： 0.9884
